In [57]:
import openai
import tiktoken
import pinecone
import os
from pymongo.mongo_client import MongoClient
import pandas as pd
from langchain.embeddings import OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()


openai.api_key = os.getenv("OPENAI_API_KEY")

PINECONE_API_KEY = 'YOUR-PINECONE-KEY'
PINECONE_API_ENV = 'YOUR-PINECONE-ENV'
MONGO_URI = os.getenv("MONGO_URI")

In [3]:
text = '''#［肥料自給　これで強化］（１）　堆肥入りペレット　まきやすく広域流通も

埼玉県加須市で水稲など５７ヘクタールを栽培する小倉祐一さん（４２）は３月、肥料成分や有機物を水田に供給するペレット状の資材を初めてまいた。資材の原料の約７割は牛ふん堆肥を中心とした国内の資源だ。小倉さんは「ブロードキャスターが使えて省力的。臭いも気にならない」という。
農水省によると、２月の肥料価格は２０２０年を１００とした指数で１５５で、前年同月を４割上回る。輸入に大きく依存する肥料原料のリン安や塩化カリの高騰が主因。こうした中で国内資源の家畜ふん堆肥への注目が高まっている。
小倉さんはリンやカリの成分量を抑えた安価なＬ型肥料への切り替えで、肥料費の圧縮を目指す。一方、減らした分のリン、カリに加え、ケイ酸、地力向上につながる有機物の補給へ、採用したペレット資材が「稲サポ」だ。１袋（１５キロ）当たりの肥料成分比はリン酸７％、カリ６％、ケイ酸８％などで、１０アール当たり５０キロを計４ヘクタールに散布した。
同省によると、家畜排せつ物は年間約８０００万トン（２０２０年）発生し、８６％が主に堆肥として活用されている。一方、稲作地帯は畜産農家が少ないなどで水田への堆肥の施用量は１５年までの３０年間で４分の１に減った。
小倉さんも、牛ふん堆肥の施用を考えたが、近隣に入手可能な農場がなかったことや散布機を持っていないこと、臭いへの懸念などから諦めてきた。稲サポのように、ペレット加工して体積を減らした資材は、堆肥では難しかった広域での流通や、従来の散布機の活用も可能になる。
稲サポを製造する朝日アグリアは、牛ふん堆肥を群馬や埼玉県の畜産農家から仕入れる。２１年に開発して以降、埼玉や茨城、千葉といった関東圏に加え、新潟などでも利用が広がる。
・地力向上し収量安定に
家畜ふん堆肥の利用拡大へ、農研機構九州沖縄農業研究センターの荒川祐介氏は「効果や価値を改めて理解する必要がある」と指摘する。ペレット資材も含め、堆肥は家畜ふんや植物の繊維などの有機物が供給できる。短期的には収益に結び付きにくいが、長期的には、有機物を微生物が分解して土中に細かな土壌の粒ができることで、排水性と保水性が向上する。収量や品質の安定につながるとする。（川崎勇）
◇
肥料価格の高騰が続いている。国内にある資源を肥料に生かす取り組みについて、普及、定着の課題を探る。


写真説明：総合１＝肥料自給　これで強化・①家畜ふん堆肥・ペレットでまきやすく・地力向上に期待・付加価値の理解必要／〇日農／営農生活部／川崎勇／堆肥入りの肥料で地力の向上に期待する小倉さん（埼玉県加須市で）


'''


In [24]:
def nouns_extraction(text, model="gpt-3.5-turbo"):
    system_message_prompt = """Your job is to pick out all proper nouns, people's names, places' name, brand names, and quoted names in Japanese for your peer translator, and also include all words where you are not confident the English translation. You will attempt to translate the words.

For companies, use the romanji name. For example, "明治生命" => "Meiji Life Insurance Co.".

You will respond in the following format:

Japanese|English|Abbreviation|Romanji|Type
農泊|Nouhaku (Agritourism)||nouhaku|Proper Noun
農水省|Ministry of Agriculture, Forestry and Fisheries|(MAFF)|nousuishou|Government Department
幸南食糧|Kounan Shokuryo Co.||kounan shokuryo|Company
鶏がゆ|chicken porridge||tori kayu|Food

"""
    message = [
        {"role": "system", "content": system_message_prompt},
        {"role": "user", "content": text},
    ]
    response = openai.ChatCompletion.create(
        model=model,
        messages=message,
        temperature=0.6,
        max_tokens=1000,
    )

    return response.choices[0].message.content

dictionary = nouns_extraction(text, "gpt-4")
print(dictionary)


埼玉県加須市|Saitama Prefecture, Kazo City||saitama ken kazoshi|Place
小倉祐一|Yuichi Ogura||ogura yuichi|Person
農水省|Ministry of Agriculture, Forestry and Fisheries|(MAFF)|nousuishou|Government Department
稲サポ|InaSapo||inasapo|Proper Noun
朝日アグリア|Asahi Aguria||asahi aguria|Company
荒川祐介|Yusuke Arakawa||arakawa yusuke|Person
川崎勇|Isamu Kawasaki||kawasaki isamu|Person
農研機構九州沖縄農業研究センター|Agriculture and Livestock Industries Corporation Kyushu Okinawa Agricultural Research Center||nouken kikou kyushu okinawa nougyou kenkyuu sentaa|Research Center
リン安|Phosphorus price||rin-an|Economic Term
塩化カリ|Potassium Chloride||enkaku kari|Chemical
ペレット状|Pellet-shaped||peretto-jou|Adjective
Ｌ型肥料|L-type fertilizer||L-gata hiryou|Fertilizer
リン酸|Phosphate||rinsan|Chemical
ケイ酸|Silicic acid||keisan|Chemical
関東圏|Kanto region||kantouken|Region
新潟|Niigata||niigata|Place
茨城|Ibaraki||ibaraki|Place
千葉|Chiba||chiba|Place
群馬|Gunma||gunma|Place



In [56]:
from supabase import create_client, Client

article_id = "28292167-fcc9-4edf-acbb-5e96c7b1af68"

url = os.getenv("SUPABASE_URL")
key = os.getenv("SUPABASE_KEY")
supabase = create_client(url, key)
for line in dictionary.strip().split('\n'):
    line = line.split("|")
    dictionary = {"jp":line[0],"en": line[1],"abbreviation": line[2],"romanji": line[3],"type": line[4]}
    data = supabase.table("article_dictionary").select("article_id,dictionary_id,dictionary(jp)").eq("dictionary.jp", line[0]).execute()
    print(line[0], data)

    if len(data.data) > 0:
        print(line[0], "already exists")
        continue
    data = supabase.table("dictionary").insert().execute()
    print(line[0], data)
    data = supabase.table("article_dictionary").insert({"article_id":article_id,"dictionary_id": data.data[0]["id"]}).execute()


埼玉県加須市 data=[{'article_id': '28292167-fcc9-4edf-acbb-5e96c7b1af68', 'dictionary_id': 49, 'dictionary': {'jp': '埼玉県加須市'}}] count=None
埼玉県加須市 already exists
小倉祐一 data=[{'article_id': '28292167-fcc9-4edf-acbb-5e96c7b1af68', 'dictionary_id': 49, 'dictionary': None}] count=None
小倉祐一 already exists
農水省 data=[{'article_id': '28292167-fcc9-4edf-acbb-5e96c7b1af68', 'dictionary_id': 49, 'dictionary': None}] count=None
農水省 already exists
稲サポ data=[{'article_id': '28292167-fcc9-4edf-acbb-5e96c7b1af68', 'dictionary_id': 49, 'dictionary': None}] count=None
稲サポ already exists
朝日アグリア data=[{'article_id': '28292167-fcc9-4edf-acbb-5e96c7b1af68', 'dictionary_id': 49, 'dictionary': None}] count=None
朝日アグリア already exists
荒川祐介 data=[{'article_id': '28292167-fcc9-4edf-acbb-5e96c7b1af68', 'dictionary_id': 49, 'dictionary': None}] count=None
荒川祐介 already exists
川崎勇 data=[{'article_id': '28292167-fcc9-4edf-acbb-5e96c7b1af68', 'dictionary_id': 49, 'dictionary': None}] count=None
川崎勇 already exists
農研機構九州沖縄農業研究セ

In [58]:
client = MongoClient(MONGO_URI)
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!
